In [1]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant import pipeline
from llm_data_quality_assistant.corruptor import RowCorruptionTypes, CellCorruptionTypes
from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np

load_dotenv()


True

In [2]:
corrupt_dataset = pd.read_csv("../datasets/parker_datasets/allergen/SHORT_allergen.csv")

gold_standard = pd.read_csv("../datasets/parker_datasets/allergen/gold_standard_alergene_pivoted.csv")

with open("../datasets/parker_datasets/allergen/allergen.partialkey", "r") as f:
    partial_keys = f.read()

with open("../datasets/parker_datasets/allergen/allergen.rules", "r") as f:
    rules = f.read()


print(partial_keys)
print(rules)
print(corrupt_dataset.head(2))
print(gold_standard.head(2))
print(type(gold_standard.get("code").iloc[0]))
print(type(corrupt_dataset.get("code").iloc[0]))

code ->
    nuts,
    almondnuts,
    brazil_nuts,
    macadamia_nuts,
    hazelnut,
    pistachio,
    walnut,
    cashew,
    celery,
    crustaceans,
    eggs,
    fish,
    gluten,
    lupin,
    milk,
    molluscs,
    mustard,
    peanut,
    sesame,
    soy,
    sulfite

-- Attribute contracts
@nuts:integer
@almondnuts:integer
@brazil_nuts:integer
@macadamia_nuts:integer
@hazelnut:integer
@pistachio:integer
@walnut:integer
@cashew:integer
@celery:integer
@crustaceans:integer
@eggs:integer
@fish:integer
@gluten:integer
@lupin:integer
@milk:integer
@molluscs:integer
@mustard:integer
@peanut:integer
@sesame:integer
@soy:integer
@sulfite:integer


The attributes
of this dataset indicate the presence (‘2’), traces (‘1’), or absence
(‘0’) of allergens in a product.

Everything value except of the code has to be between 0 and 2.

--NOT ALLOWED THAT
nuts < almondnuts
nuts < brazil_nuts
nuts < macadamia_nuts
nuts < hazelnut
nuts < pistachio
nuts < walnut
nuts < cashew

IT ALLWAYS MUST BE

In [3]:
shortened_corrupt_df = corrupt_dataset[corrupt_dataset["code"].isin(gold_standard["code"])]
shortened_corrupt_df = shortened_corrupt_df.sort_values(by="code").reset_index(drop=True)

print("Shape shortened corrupt dataset:")
print(shortened_corrupt_df.shape)

print(shortened_corrupt_df)

shortened_gold_standard = gold_standard[gold_standard["code"].isin(shortened_corrupt_df["code"])]
shortened_gold_standard = shortened_gold_standard.sort_values(by="code").reset_index(drop=True)

print("Shape shortened gold standard dataset:")
print(shortened_gold_standard.shape)

print(shortened_gold_standard)


Shape shortened corrupt dataset:
(18, 22)
             code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
0   4104420006065     0           0            0               0         0   
1   4104420006065     0           0            0               0         0   
2   4104420007963     2           0            0               0         0   
3   4104420007963     2           0            0               0         2   
4   4104420007987     2           0            0               0         0   
5   4104420007987     2           0            0               0         2   
6   4104420010628     0           0            0               0         0   
7   4104420010628     2           0            0               0         0   
8   4104420014701     0           0            0               0         0   
9   4104420014701     0           0            0               0         0   
10  4104420015289     1           0            0               0         0   
11  4104420015289     

In [4]:
# Clean and evaluate using the new Pipeline API
from llm_data_quality_assistant.pipeline import Pipeline
from llm_data_quality_assistant.enums import Models

# Use a primary key for merging
primary_key = "code"
model = Models.GeminiModels.GEMINI_2_0_FLASH_LITE
rpm = 30

# Merge/clean with LLM
merged_df = Pipeline.merge_with_llm(
    dataset=shortened_corrupt_df,
    primary_key=primary_key,
    model_name=model,
    rpm=rpm,
    additional_prompt=rules,
    verbose=True
)

# Evaluate results
stats_micro = Pipeline.evaluate_micro(
    gold_standard=shortened_gold_standard,
    cleaned_dataset=merged_df,
    corrupted_dataset=shortened_corrupt_df
)
pprint(stats_micro)

# Show DataFrames for inspection
print("Merged DataFrame:")
print(merged_df)
print("Gold Standard DataFrame:")
print(shortened_gold_standard)
print("Corrupted DataFrame:")
print(shortened_corrupt_df)

[
  {
  {
    "code": 4104420006
    "code": 4104420006065,
    "nuts": 0,
    "al065,
    "nuts": 0,
    "almondnuts": 0,
    "brazil_nuts": 0,
    "macadamia_nuts": 0,
    "hazmondnuts": 0,
    "brazil_nuts": 0,
    "macadamia_nuts": 0,
    "hazelnut": 0,
    "pistachio": 0,
    "walnut": 0,
    "cashew": 0elnut": 0,
    "pistachio": 0,
    "walnut": 0,
    "cashew": 0,
    "celery": 0,
    "crustaceans": 0,
    "eggs": 1,
    "fish": 0,
    "gluten": 2,
    "l,
    "celery": 0,
    "crustaceans": 0,
    "eggs": 1,
    "fish": 0,
    "gluten": 2,
    "lupin": 0,
    "milk": 0,
    "molluscs": 0,
    "mustard": 0,
    "peanut": 0,
    "sesame":upin": 0,
    "milk": 0,
    "molluscs": 0,
    "mustard": 0,
    "peanut": 0,
    "sesame": 0,
    "soy": 1,
    "sulfite": 0
  }
] 0,
    "soy": 1,
    "sulfite": 0
  }
][[
  {
  {
    "code": 4104420007
    "code": 4104420007963,
    "nuts": 2,
    "al963,
    "nuts": 2,
    "almondnuts": 0,
    "brazil_nuts": 0,
    "macadamia_nuts": 0,
    

In [5]:
print("Shortened Merged DataFrame:")
print(shortened_merged_df)
print("Shortened Gold Standard DataFrame:")
print(shortened_gold_standard)

Shortened Merged DataFrame:


NameError: name 'shortened_merged_df' is not defined